<a href="https://www.kaggle.com/code/yashnikam10/finetunebloomz-mt-0-model?scriptVersionId=153969106" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/anlp-project-dataset/prompts.json
/kaggle/input/pls-work-15/prompts15.json
/kaggle/input/anlp-project-prompt-5/prompts1.json
/kaggle/input/hey-maybe-this-will-work/prompts3.json


In [2]:
!pip install datasets==2.13.1
!pip install transformers==4.34.1
!pip install accelerate
!pip install peft==0.5.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 11.3 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 11.9 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 13.1 MB/s eta 0:00:00
  Attempting uninstall: dill
    Found existing installation: dill 0.3.7
    Uninstalling dill-0.3.7:
      Successfully uninstalled dill-0.3.7
  Attempting uninstall: multiprocess
    Found existing installation: multiprocess 0.70.15
    Uninstalling multiprocess-0.70.15:
      Successfully uninstalled multiprocess-0.70.15
  Attempting uninstall: datasets
    Found existing installation: datasets 2.1.0
    Uninstalling datasets-2.1.0:
      Successfully uninstalled datasets-2.1.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behav

In [3]:
import datasets
import json
import torch
import pandas as pd
import transformers
from transformers import BloomTokenizerFast, BloomForCausalLM, TrainingArguments, Trainer
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, Dataset
from accelerate import Accelerator

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [6]:
checkpoint = "bigscience/mt0-base"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint, torch_dtype="auto", device_map="auto")

In [7]:
# import os 
# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "caching-allocator"

In [8]:
import transformers
data_collator = transformers.DataCollatorForTokenClassification(tokenizer=tokenizer)

In [9]:
dataset = load_dataset("json", data_files="/kaggle/input/pls-work-15/prompts15.json")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-1cb6d37db145a303/0.0.0/8bb11242116d547c741b2e8a1f18598ffdd40a1d4f2a2872c7a28b697434bc96. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
# !pip install -U datasets

In [11]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 15
    })
})

In [12]:
# tokenizer.max_length()

In [13]:
def prepare_train_data(data):
    # prompt + completion
    #text_input = data['prompt'] + ' ' + data['completion']
    text_input = data['text']
    # tokenize the input (prompt + completion) text
    tokenized_input = tokenizer(text_input, return_tensors='pt', padding='max_length', truncation=True, max_length=2500)
    # generative models: labels are the same as the input
    tokenized_input['labels'] = tokenized_input['input_ids']
    return tokenized_input

In [14]:
train_dataset = dataset.map(prepare_train_data,batched=True, remove_columns=["text"])

Map:   0%|          | 0/15 [00:00<?, ? examples/s]

In [15]:
training_arguments = TrainingArguments(
    '/kaggle/working/HindiQA-bloomz-560m',
    learning_rate=1e-4,
    per_device_train_batch_size=1,
    num_train_epochs=8,
    weight_decay=0.01,
    optim="adafactor",
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
)

In [16]:
trainer = Trainer(
    model = model,
    args = training_arguments,
    train_dataset = train_dataset["train"],
    tokenizer=tokenizer,
    data_collator=data_collator
)

In [17]:
trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...


Step,Training Loss


TrainOutput(global_step=24, training_loss=10.981741587320963, metrics={'train_runtime': 326.9812, 'train_samples_per_second': 0.367, 'train_steps_per_second': 0.073, 'total_flos': 562052874240000.0, 'train_loss': 10.981741587320963, 'epoch': 6.4})

In [18]:
trainer.save_model()

In [19]:
tokenizer = AutoTokenizer.from_pretrained("/kaggle/working/HindiQA-bloomz-560m")

In [20]:
model = AutoModelForSeq2SeqLM.from_pretrained("./HindiQA-bloomz-560m")

In [21]:
essay = '"दलाई लामा ने अपनी संवेदना और प्रार्थना व्यक्त की और कलाम की मौत को "एक अपूरणीय क्षति" बुला, अपना दुख व्यक्त किया। उन्होंने यह भी कहा, "अनेक वर्षों में, मुझे कई अवसरों पर कलाम के साथ बातचीत करने का मौका मिला। वह एक महान वैज्ञानिक, शिक्षाविद और राजनेता ही नहीं, बल्कि वे एक वास्तविक सज्जन थे, और हमेशा मैंने उनकी सादगी और विनम्रता की प्रशंसा की है। मैंने सामान्य हितों के विषयों की एक विस्तृत श्रृंखला पर हमारी चर्चाओं का आनंद लिया, लेकिन विज्ञान, अध्यात्म और शिक्षा के साथ मुख्य रूप से हमारे बीच चिंतन किया जाता था। " दक्षिण एशियाई नेताओं ने अपनी संवेदना व्यक्त की और दिवंगत राजनेता की सराहना की। भूटान सरकार ने कलाम की मौत के शोक के लिए देश के झंडे को आधी ऊंचाई पर फहराने के लिए आदेश दिया, और श्रद्धांजलि में 1000 मक्खन के दीपक की भेंट किए। भूटान के प्रधानमंत्री शेरिंग तोबगे ने कलाम के प्रति अपना गहरा दुख व्यक्त करते हुए कहा, "वे एक महान नेता थे जिनकी सभी ने प्रशंसा की विशेषकर भारत के युवाओं के वे प्रशंसनीय नेता थे जिन्हें वे जनता का राष्ट्रपति बुलाते थे। "बांग्लादेश की प्रधानमंत्री शेख हसीना ने उनकी व्याख्या करते हुए कहा, "एक महान राजनेता प्रशंसित वैज्ञानिक और दक्षिण एशिया के युवा पीढ़ी के लिए प्रेरणा स्रोत के संयोग" उन्होंने कलाम की मृत्यु को "भारत के लिए अपूरणीय क्षति से भी परे बताया। " उन्होंने यह भी कहा कि भारत के सबसे प्रसिद्ध बेटे, पूर्व राष्ट्रपति के निधन पर हमें गहरा झटका लगा है। ए॰पी॰जे॰ अब्दुल कलाम अपने समय के सबसे महान ज्ञानियों में से एक थे। वह बांग्लादेश में भी बहुत सम्मानित थे। उनकी विज्ञान और प्रौद्योगिकी के क्षेत्र में भारत की वृद्धि करने के लिए अमूल्य योगदान के लिए वे सभी के द्वारा हमेशा याद किये जायेंगे। वे दक्षिण एशिया की युवा पीढ़ी के लिए प्रेरणा का स्रोत थे जो उनके सपनों को पंख देते थे। " बांग्लादेश नेशनलिस्ट पार्टी की प्रमुख खालिदा जिया ने कहा, "एक परमाणु वैज्ञानिक के रूप में, उन्होंने लोगों के कल्याण में स्वयं को समर्पित किया। "अफगानिस्तान के राष्ट्रपति अशरफ गनी, ने कलाम को, "लाखों लोगों के लिए एक प्रेरणादायक शख्सियत बताया" ये नोट करते हुए "हमे अपने जीवन से बहुत कुछ सीखना है। " नेपाली प्रधानमंत्री सुशील कोइराला ने भारत के लिए कलाम के वैज्ञानिक योगदानों को याद किया। "नेपाल ने एक अच्छा दोस्त खो दिया है और मैंने एक सम्मानित और आदर्श व्यक्तित्व को खो दिया है। "पाकिस्तान के राष्ट्रपति, ममनून हुसैन और पाकिस्तान के प्रधानमंत्री नवाज शरीफ ने पूर्व राष्ट्रपति के निधन पर उनके प्रति दु: ख, शोक व संवेदना व्यक्त की। श्रीलंका के राष्ट्रपति मैत्रिपाला सिरिसेना ने कहा, "कलाम दृढ़ विश्वास और अदम्य भावना के आदमी थे। मैंने उन्हें दुनिया के एक उत्कृष्ट राजनेता के रूप में देखा था। उनकी मौत भारत के लिए, बल्कि पूरी दुनिया के लिए अपूरणीय क्षति है। "इंडोनेशियाई राष्ट्रपति सुसीलो बम्बनग युधोयोनो, मलेशिया के प्रधानमंत्री नजीब रजाक, सिंगापुर के प्रधानमंत्री ली सियन लूंग , संयुक्त अरब अमीरात के राष्ट्रपति शेख खलीफा बिन जायद अल नहयान सहित अन्य अंतरराष्ट्रीय नेताओं,, और संयुक्त अरब अमीरात के प्रधानमंत्री और दुबई के शासक ने भी कलाम को श्रद्धांजलि अर्पित की। रूसी राष्ट्रपति व्लादिमीर पुतिन ने भारत सरकार, भारत के सभी लोगों के लिए और मृतक नेता ले प्रियजनों के लिए अपनी गंभीर संवेदना व्यक्त की और अपनी सहानुभूति और समर्थन से अवगत कराते हुए कहा, "कलाम को हमारे देशों के बीच लगातार मैत्रीपूर्ण संबंधों के एक प्रतिपादक के रूप में याद किया जाएगा, उन्होंने भारत की राष्ट्रीय सुरक्षा को सुनिश्चित करने में सामाजिक, आर्थिक, वैज्ञानिक और तकनीकी प्रगति के लिए व्यक्तिगत योगदान दिया। उन्होंने  पारस्परिक रूप से लाभप्रद रूसी-भारतीय सहयोग जोड़ने के लिए बहुत कुछ किया। "संयुक्त राज्य अमेरिका के राष्ट्रपति बराक ओबामा ने कहा," अमेरिकी लोगों की ओर से, मैं पूर्व भारतीय राष्ट्रपति ए॰पी॰जे॰ अब्दुल कलाम के निधन पर भारत के लोगों के लिए अपनी गहरी संवेदना का विस्तार करना चाहता हूँ। एक वैज्ञानिक और राजनेता, कलाम ने अपनी विनम्रता से घर में और विदेशों में सम्मान कमाया और भारत के सबसे महान नेताओं में से एक बने। भारत-अमेरिका के मजबूत संबंधों के लिए, डा कलाम ने सदा वकालत की। 1962 में संयुक्त राज्य अमेरिका की यात्रा के दौरान नासा के साथ अंतरिक्ष सहयोग को गहरा करने के लिए काम किया।'

In [22]:
prompt = 'Given the context delimited by triple backticks ```'+ essay + '```, generate some questions from the given context. Questions:'
generator = transformers.pipeline('text2text-generation', model=model, tokenizer=tokenizer,do_sample=False)
result = generator(prompt, max_new_tokens=128)
print(result)

[{'generated_text': "'कलाम दृढ़ विश्वास और अदम्य भावना के आदमी थे'"}]


In [23]:
question = result[0]['generated_text']
prompt2 = 'Given the context delimited by triple backticks ```'+ essay + '```,and the question delimited by single backticks `'+ question + '` generate an answer.Answer:'

In [24]:
generator = transformers.pipeline('text2text-generation', model=model, tokenizer=tokenizer,do_sample=False)
result = generator(prompt2, max_new_tokens=128)
print(result)

[{'generated_text': "'कलाम दृढ़ विश्वास और अदम्य भावना के आदमी थे'"}]


In [25]:
essay = 'पेड़ों को हरा सोना भी कहा जाता है क्योंकि यह बहुत मूल्यवान सम्पदा है। धरती पर जीवन प्रदान करने वाली ऑक्सीजन और पानी प्रदान करने वाला मुख्य साधन पेड़ ही है। ऑक्सीजन प्रदान करने का कार्य और कोई नहीं कर सकता और पेड़ों के बिना पानी की कल्पना करना मुश्किल ही नहीं, नामुमकिन है।पेड़ वायु प्रदूषण कम करने में हमारी सहायता कर पर्यावरण को शुद्ध रखते हैं। मात्र वायु प्रदूषण ही नहीं ये, हानिकारक रसायनों का छानकर जल को भी साफ करते हैं। हर उद्योग में पेड़ के उत्पाद का मुख्य योगदान रहता है। हमारे दैनिक जीवन में उपयोग होने वाली वस्तुओं में पेड़ों का बहुत महत्व है।जितने अधिक पेड़ होंगे पर्यावरण भी उतना ही शुद्ध रहेगा। आजकल लोगों को वायु प्रदूषण के कारण कई प्रकार के साँस के एवं अन्य रोगों से पीड़ित होना पड़ रहा है।यदि पेड़ होंगे तो हवा में मिली हानिकारक गैसों को शोषित कर हमें स्वच्छ हवा प्रदान करेंगे और रोगों से छुटकारा भी। यदि हम पेड़ों की संख्या में वृद्धि करेंगे तो ये प्राकृतिक रुप से हवा को स्वच्छ करने के साथ हमें और भी कई फायदे पहुँचायेंगे। इनकी वृद्धि से हम एयर कंडीशनर के उपयोग से बच कर इससे निकलने वाली हानिकारक गैसों से भी निजात दिलाते हैं। पेड़ के कारण ही हमें भरपूर वर्षा प्राप्त होती है।पेड़ की जड़े मिट्टी को बांध कर रखती हैं जिनसे भूमि कटाव भी नहीं होता व भूमि जल को अच्छे से अवशोषित कर लेती है। यही जल भूमिगत जल बनकर हमें मनुष्य में पानी के अभाव से बचाता है। पेड़ हमें छाया प्रदान कर गर्मी के प्रभाव से भी धरती को बचाते हैं।इस अमूल्य सम्पदा की कमी से धरती पर ग्लोबल वार्मिंग, सूखा, भूमि कटाव जैसे समस्यायें अपना विकराल रूप लेती जा रही हैं। यदि हम इस प्राकृतिक आपदाओं से बचना चाहते हैं तो हमें पेड़ों के संरक्षण की ओर कदम उठाने ही होंगे।यदि आज हम इस दिशा में कार्य करेंगे तभी भावी पीढ़ी को भी इस ओर काम करने की प्रेरणा मिलेगी। ख़ुशी के अवसर पर हम पैसा खर्च करते हैं, दावतें करते हैं लेकिन इन सब के बजाय हम पौधारोपण और पेड़ों का संरक्षण करें तो ये सिर्फ हमारे जान-पहचान वालों के लिए ही नहीं बल्कि मनुष्य के लिए भी ख़ुशी का संकेत होगी।'

In [26]:
prompt = 'Given the context delimited by triple backticks ```'+ essay + '```, generate some questions from the given context. Questions:'
generator = transformers.pipeline('text2text-generation', model=model, tokenizer=tokenizer,do_sample=False)
result = generator(prompt, max_new_tokens=128)
print(result)

[{'generated_text': 'पेड़ों का संरक्षण करने के लिए हम कदम उठाने ही होंगे?'}]


In [27]:
question = result[0]['generated_text']
prompt2 = 'Given the context delimited by triple backticks ```'+ essay + '```,and the question delimited by single backticks `'+ question + '` generate an answer.Answer:'

In [28]:
generator = transformers.pipeline('text2text-generation', model=model, tokenizer=tokenizer,do_sample=False)
result = generator(prompt2, max_new_tokens=128)
print(result)

[{'generated_text': 'यदि हम इस प्राकृतिक आपदाओं से बचना चाहते हैं तो हमें पेड़ों के संरक्षण की ओर कदम उठाने ही होंगे।'}]


In [29]:
essay = 'अकबर के हिन्दू सामंत उसकी अनुमति के बगैर मंदिर निर्माण तक नहीं करा सकते थे। बंगाल में राजा मानसिंह ने एक मंदिर का निर्माण बिना अनुमति के आरंभ किया, तो अकबर ने पता चलने पर उसे रुकवा दिया और १५९५ में उसे मस्जिद में बदलने के आदेश दिए। अकबर के लिए आक्रोश की हद एक घटना से पता चलती है। हिन्दू किसानों के एक नेता राजा राम ने अकबर के मकबरे, सिकंदरा, आगरा को लूटने का प्रयास किया, जिसे स्थानीय फ़ौजदार, मीर अबुल फजल ने असफल कर दिया। इसके कुछ ही समय बाद १६८८ में राजा राम सिकंदरा में दोबारा प्रकट हुआ और शाइस्ता खां के आने में विलंब का फायदा उठाते हुए, उसने मकबरे पर दोबारा सेंध लगाई और बहुत से बहुमूल्य सामान, जैसे सोने, चाँदी, बहुमूल्य कालीन, चिराग, इत्यादि लूट लिए, तथा जो ले जा नहीं सका, उन्हें बर्बाद कर गया। राजा राम और उसके आदमियों ने अकबर की अस्थियों को खोद कर निकाल लिया एवं जला कर भस्म कर दिया, जो कि मुस्लिमों के लिए घोर अपमान का विषय था। बाद के वर्षों में अकबर को अन्य धर्मों के प्रति भी आकर्षण हुआ। अकबर का हिंदू धर्म के प्रति लगाव केवल मुग़ल साम्राज्य को ठोस बनाने के ही लिए नही था वरन उसकी हिंदू धर्म में व्यक्तिगत रुचि थी। हिंदू धर्म के अलावा अकबर को शिया इस्लाम एवं ईसाई धर्म में भी रुचि थी। ईसाई धर्म के मूलभूत सिद्धांत जानने के लिए उसने एक बार एक पुर्तगाली ईसाई धर्म प्रचारक को गोआ से बुला भेजा था। अकबर ने दरबार में एक विशेष जगह बनवाई थी जिसे इबादत-खाना (प्रार्थना-स्थल) कहा जाता था, जहाँ वह विभिन्न धर्मगुरुओं एवं प्रचारकों से धार्मिक चर्चाएं किया करता था। उसका यह दूसरे धर्मों का अन्वेषण कुछ मुस्लिम कट्टरपंथी लोगों के लिए असहनीय था। उन्हे लगने लगा था कि अकबर अपने धर्म से भटक रहा है। इन बातों में कुछ सच्चाई भी थी, अकबर ने कई बार रुढ़िवादी इस्लाम से हट कर भी कुछ फैसले लिए, यहाँ तक कि १५८२ में उसने एक नये सम्प्रदाय की ही शुरुआत कर दी जिसे दीन-ए-इलाही यानी ईश्वर का धर्म कहा गया। दीन-ए-इलाही नाम से अकबर ने १५८२ में एक नया धर्म बनाया जिसमें सभी धर्मो के मूल तत्वों को डाला, इसमे प्रमुखतः हिंदू एवं इस्लाम धर्म थे। इनके अलावा पारसी, जैन एवं ईसाई धर्म के मूल विचारों को भी सम्मिलित किया। हालांकि इस धर्म के प्रचार के लिए उसने कुछ अधिक उद्योग नहीं किये केवल अपने विश्वस्त लोगो को ही इसमे सम्मिलित किया। कहा जाता हैं कि अकबर के अलावा केवल राजा बीरबल ही मृत्यु तक इस के अनुयायी थे। दबेस्तान-ए-मजहब के अनुसार अकबर के पश्चात केवल १९ लोगो ने इस धर्म को अपनाया। कालांतर में अकबर ने एक नए पंचांग की रचना की जिसमे कि उसने एक ईश्वरीय संवत को आरम्भ किया जो उसके ही राज्याभिषेक के दिन से प्रारम्भ होता था। उसने तत्कालीन सिक्कों के पीछे ‘‘अल्लाह-ओ-अकबर’’ लिखवाया जो अनेकार्थी शब्द था। अकबर का शाब्दिक अर्थ है "महान" और ‘‘अल्लाह-ओ-अकबर’’ शब्द के दो अर्थ हो सकते थे "अल्लाह महान हैं " या "अकबर ही अल्लाह हैं"। दीन-ए-इलाही सही मायनो में धर्म न होकर एक आचार संहिता के समान था। इसमे भोग, घमंड, निंदा करना या दोष लगाना वर्जित थे एवं इन्हे पाप कहा गया। दया, विचारशीलता और संयम इसके आधार स्तम्भ थे। यह तर्क दिया गया है कि दीन-ए-इलैही का एक नया धर्म होने का सिद्धांत गलत धारणा है, जो कि बाद में ब्रिटिश इतिहासकारों द्वारा अबुल फजल के कार्यों के गलत अनुवाद के कारण पैदा हुआ था। हालांकि, यह भी स्वीकार किया जाता है कि सुलह-ए-कुल की नीति, जिसमें दीन-ई-इलैही का सार था, अकबर ने केवल धार्मिक उद्देश्यों के लिए नहीं बल्कि सामान्य शाही प्रशासनिक नीति का एक भाग के रूप में अपनाया था। इसने अकबर की धार्मिक सहानुभूति की नीति का आधार बनाया। 1605 में अकबर की मौत के समय उनके मुस्लिम विषयों में असंतोष का कोई संकेत नहीं था, और अब्दुल हक जैसे एक धर्मशास्त्री की धारणा थी कि निकट संबंध बने रहे। र्वधर्म मैत्री सुलह-ए-कुल सूफियों का मूल सिद्धांत रहा है। प्रसिद्ध सूफी शायर रूमी के पास एक व्यक्ति आया और कहने लगा- एक मुसलमान एक ईसाई से सहमत नहीं होता और ईसाई यहूदी से।'

In [30]:
# for i in range(1):
prompt = 'Given the context delimited by triple backticks ```'+ essay + '```, generate some questions from the given context. Questions:'
#     input_ids = tokenizer.encode(prompt, return_tensors='pt')
#     greedy_output = model.generate(input_ids, num_beams = 4,no_repeat_ngram_size=4,min_length=5,max_new_tokens=100)
#     print(tokenizer.decode(greedy_output[0], skip_special_tokens=True))
#     question = tokenizer.decode(greedy_output[0], skip_special_tokens=True)
#     prompt2 = 'Given the context delimited by triple backticks ```{'+ essay + '}```,and the question delimited by single backticks `{'+ question + '}` generate an answer.Answer:'
#     input_ids = tokenizer.encode(prompt2, return_tensors='pt')
#     greedy_output = model.generate(input_ids, num_beams=4, no_repeat_ngram_size=2, max_new_tokens=100,do_sample=True)
#     print(tokenizer.decode(greedy_output[0], skip_special_tokens=True))

In [31]:
# !zip -r bloomz.zip /kaggle/working/HindiQA-bloomz-560m

In [32]:
generator = transformers.pipeline('text2text-generation', model=model, tokenizer=tokenizer,do_sample=False)
result = generator(prompt, max_new_tokens=128)
print(result)

[{'generated_text': 'अकबर के हिन्दू सामंत किस कारण मंदिर निर्माण तक नहीं करा सकते थे?'}]


In [33]:
question = result[0]['generated_text']

In [34]:
prompt2 = 'Given the context delimited by triple backticks ```'+ essay + '```,and the question delimited by single backticks `'+ question + '` generate an answer.Answer:'

In [35]:
generator = transformers.pipeline('text2text-generation', model=model, tokenizer=tokenizer,do_sample=False)
result = generator(prompt2, max_new_tokens=128)
print(result)

[{'generated_text': 'उसकी अनुमति के बगैर'}]
